<a href="https://colab.research.google.com/github/coatless/colab-notes/blob/main/01-pandas-to-r-data-types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# (Aside) Fix rpy2 version

Downgrade to address [Conversion 'py2rpy' not defined for objects of type '<class 'str'>'](https://stackoverflow.com/questions/74283327/conversion-py2rpy-not-defined-for-objects-of-type-class-str)

In [40]:
# Downgrade rpy2 from 3.5.5
!pip install rpy2==3.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 KB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rpy2: filename=rpy2-3.5.1-cp38-cp38-linux_x86_64.whl size=310202 sha256=89957bc63dad301d14886c402616e4d9f56c5b834a8a392084b182ca75297f8c
  Stored in directory: /root/.cache/pip/wheels/6b/40/7d/f63e87fd83e8b99ee837c8e3489081c4b3489134bc520235ed
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.5.5
    Uninstalling rpy2-3.5.5:
      Successfully uninstalled rpy2-3.5.5


# Coversion of Pandas to R Data Types

Create a data frame in Pandas and observe types from different variable initializations.

https://pandas.pydata.org/docs/user_guide/basics.html#basics-dtypes

In [6]:
import pandas as pd 
import numpy as np 

dft = pd.DataFrame(
    {
        # Scalars
        "integer": 1,
        "numeric": 3.14,
        "logical": False,
        "character": "foo",
        "complex": complex(1, 2),
        # Series
        "numeric-list": pd.Series([1.0] * 3).astype("float32"),
        "integer-list": pd.Series([1] * 3, dtype="int8"),
        "complex-list": pd.Series(np.array([1, 2, 3]) + np.array([4, 5, 6]) *1j).astype("complex128"),
        "character-list": pd.Series(["hello", "world", "stat"]),
        "logical-list": pd.Series([True, False, True]),
        # Time Dependency: https://pandas.pydata.org/docs/user_guide/timeseries.html
        "POSIXct-POSIXt-timestamp": pd.Timestamp("20230102"),
        "POSIXct-POSIXt-date_range": pd.date_range("2023", freq="D", periods=3),
        #"POSIXct-POSIXt-period": pd.period_range("1/1/2011", freq="M", periods=3), # Not supported in rpy2
        #"POSIXct-POSIXt-timedelta": pd.to_timedelta(np.arange(3), unit="s"), # Not supported in rpy2
        # Categorical: https://pandas.pydata.org/docs/user_guide/categorical.html
        "factor": pd.Categorical(["a", "b", "c"], ordered=False),
        "ordered-factor": pd.Categorical(["a", "b", "c"], categories=["a", "b", "c"], ordered=True),
    }
)

dft

,integer,numeric,logical,character,complex,numeric-list,integer-list,complex-list,character-list,logical-list,POSIXct-POSIXt-timestamp,POSIXct-POSIXt-date_range,factor,ordered-factor
0,1,3.14,False,foo,1.0+2.0j,1.0,1,1.0+4.0j,hello,True,2023-01-02,2023-01-01,a,a
1,1,3.14,False,foo,1.0+2.0j,1.0,1,2.0+5.0j,world,False,2023-01-02,2023-01-02,b,b
2,1,3.14,False,foo,1.0+2.0j,1.0,1,3.0+6.0j,stat,True,2023-01-02,2023-01-03,c,c


In [2]:
dft.dtypes

integer                               int64
numeric                             float64
logical                                bool
character                            object
complex                          complex128
numeric-list                        float32
integer-list                           int8
complex-list                     complex128
character-list                       object
logical-list                           bool
POSIXct-POSIXt-timestamp     datetime64[ns]
POSIXct-POSIXt-date_range    datetime64[ns]
factor                             category
ordered-factor                     category
dtype: object

Check by passing the data frame into _R_ using `rpy2`:

In [3]:
%load_ext rpy2.ipython

In [5]:
%%R -i dft

sapply(dft, class)

$integer
[1] "integer"

$numeric
[1] "numeric"

$logical
[1] "logical"

$character
[1] "character"

$complex
[1] "complex"

$`numeric-list`
[1] "numeric"

$`integer-list`
[1] "integer"

$`complex-list`
[1] "complex"

$`character-list`
[1] "character"

$`logical-list`
[1] "logical"

$`POSIXct-POSIXt-timestamp`
[1] "POSIXct" "POSIXt" 

$`POSIXct-POSIXt-date_range`
[1] "POSIXct" "POSIXt" 

$factor
[1] "factor"

$`ordered-factor`
[1] "ordered" "factor" 

